In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
from scipy.misc import imread, imresize, imsave
import tensorflow.contrib.layers as layers

GPUID = 0
os.environ["CUDA_VISIBLE_DEVICES"] = str(GPUID)
__author__ = 'qi.wei@siemens.com'

mb_size = 144
X_dim = 784
Z_dim = 16
h_dim = 128

mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
tf.reset_default_graph()  

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
def save_model(model_path, saver, sess, step):
    """
    save model with path error checking
    """
    if model_path is None:
        my_path = "model/myckpt" # default path in tensorflow saveV2 format
        # try to make directory
        if not os.path.exists("model"):
            try:
                os.makedirs("model")
            except OSError as e:
                if e.errno != errno.EEXIST:
                    raise
    else: 
        my_path = model_path
    saver.save(sess, my_path, global_step=step)
        
def plot(samples,n):
    fig = plt.figure(figsize=(n, n))
    gs = gridspec.GridSpec(n, n)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    return fig

In [3]:
def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)

def weight_variable(shape):  
    initial = tf.truncated_normal(shape,stddev=0.1) #截断正态分布，此函数原型为尺寸、均值、标准差  
    return tf.Variable(initial)  
def bias_variable(shape):  
#     initial = tf.constant(0.1,shape=shape)  
    initial = tf.constant(0.0,shape=shape)  
    return tf.Variable(initial)  

In [4]:
X = tf.placeholder(tf.float32, shape=[None, X_dim])

# D_W1 = tf.Variable(xavier_init([X_dim, h_dim]))
# D_b1 = tf.Variable(tf.zeros(shape=[h_dim]))

# D_W2 = tf.Variable(xavier_init([256, 128]))
# D_b2 = tf.Variable(tf.zeros(shape=[1]))

# D_W3 = tf.Variable(xavier_init([h_dim, 1]))
# D_b3 = tf.Variable(tf.zeros(shape=[1]))
with tf.variable_scope("discriminator"):
    DW_conv1 = weight_variable([3, 3, 1, 32])
    Db_conv1 = bias_variable([32])

    DW_conv2 = weight_variable([3, 3, 32, 64])
    Db_conv2 = bias_variable([64])

    DW_conv3 = weight_variable([3, 3, 64, 128])
    Db_conv3 = bias_variable([128])
    
    DW_conv4 = weight_variable([3, 3, 128, 1])
    Db_conv4 = bias_variable([1])

#     DW_fc4 = weight_variable([3*3*128,128])  
#     Db_fc4 = bias_variable([128])  

#     DW_fc5 = weight_variable([128,1])  
#     Db_fc5 = bias_variable([1])

    #theta_D = [D_W1, D_W2, D_W3, D_b1, D_b2, D_b3]
    theta_D = [DW_conv1, DW_conv2, DW_conv3, DW_conv4, Db_conv1, Db_conv2, Db_conv3, Db_conv4] #DW_fc4, DW_fc5, Db_fc4, Db_fc5

In [5]:
Z = tf.placeholder(tf.float32, shape=[None, 1, 1, Z_dim])

# G_W1 = tf.Variable(xavier_init([Z_dim, h_dim]))
# G_b1 = tf.Variable(tf.zeros(shape=[h_dim]))

# G_W2 = tf.Variable(xavier_init([128, 256]))
# G_b2 = tf.Variable(tf.zeros(shape=[256]))

# G_W3 = tf.Variable(xavier_init([h_dim, X_dim]))
# G_b3 = tf.Variable(tf.zeros(shape=[X_dim]))

# theta_G = [G_W1, G_b1, G_W2, G_b2, G_W3, G_b3]

with tf.variable_scope("generator"):
#     GW_fc1 = weight_variable([Z_dim,h_dim])
#     Gb_fc1 = bias_variable([h_dim])

#     GW_fc2 = weight_variable([h_dim,7*7*128])
#     Gb_fc2 = bias_variable([7*7*128])
    
    GW_conv1 = weight_variable([3, 3, 64, Z_dim])
    Gb_conv1 = bias_variable([64])
    
    GW_conv2 = weight_variable([3, 3, 32, 64])
    Gb_conv2 = bias_variable([32])

# GW_conv3 = weight_variable([1, 1, 256, 256])
# Gb_conv3 = bias_variable([256])

    GW_conv4 = weight_variable([3, 3, 16, 32])
    Gb_conv4 = bias_variable([16])

    GW_conv5 = weight_variable([3, 3, 1, 16])
    Gb_conv5 = bias_variable([1])

    theta_G = [GW_conv1, Gb_conv1, GW_conv2, Gb_conv2, GW_conv4, Gb_conv4, GW_conv5, Gb_conv5] #  GW_conv3, Gb_conv3,  GW_fc1, Gb_fc1, GW_fc2, Gb_fc2

In [6]:
def sample_Z(m, n):
    return np.random.uniform(-1., 1., size=[m, 1, 1, n])

In [7]:
def generator(z):
#     G_h1 = tf.nn.relu(tf.matmul(z, G_W1) + G_b1)
#     #G_h2 = tf.nn.relu(tf.matmul(G_h1, G_W2) + G_b2)
#     G_log_prob = tf.matmul(G_h1, G_W3) + G_b3
#     G_prob = tf.nn.sigmoid(G_log_prob)

#     G_h1 = tf.nn.relu(tf.matmul(z,GW_fc1) + Gb_fc1)
#     G_h2 = tf.matmul(G_h1,GW_fc2) + Gb_fc2
#     h_pool_tensor = tf.reshape(G_h2, [-1,7,7,128])
#     #h_pool_tensor = tf.reshape(G_h2, [-1,28*28*1])
#     print h_pool_tensor.get_shape()
#     bs = tf.shape(h_pool_tensor)[0]

    bs = tf.shape(z)[0]
    G_h1 = tf.nn.relu(tf.nn.conv2d_transpose(z, GW_conv1, output_shape=[bs, 3, 3, 64], 
                                                               strides=[1, 2, 2, 1], padding='VALID') + Gb_conv1)
    G_h2 = tf.nn.relu(layers.batch_norm(tf.nn.conv2d_transpose(G_h1, GW_conv2, output_shape=[bs,7,7,32], 
                                                               strides=[1, 2, 2, 1], padding='VALID') + Gb_conv2))

#     G_h3 = tf.nn.relu(tf.nn.conv2d_transpose(h_pool_tensor, GW_conv3, output_shape=[bs,7,7,256], strides=[1, 1, 1, 1], padding='VALID') + Gb_conv3)
#     print G_h3.get_shape()
    G_h4 = tf.nn.relu(layers.batch_norm(tf.nn.conv2d_transpose(G_h2, GW_conv4, output_shape=[bs,14,14,16], 
                                                               strides=[1, 2, 2, 1], padding='SAME') + Gb_conv4))
    print G_h4.get_shape()
    G_log_prob = tf.nn.conv2d_transpose(G_h4, GW_conv5, output_shape=[bs,28,28,1],
                                                          strides=[1, 2, 2, 1], padding='SAME') + Gb_conv5
    G_prob = tf.nn.sigmoid(tf.reshape(G_log_prob, [-1,28*28*1]))
    print G_prob.get_shape()
    

#     bs = tf.shape(x)[0]
#     Bn = True
#     deconv0 = deconv2d(z, [bs, 4, 4, Cc], 4, 4, 1, 1, bias=not Bn, padding='VALID')
#     deconv0 = tf.nn.relu(layers.batch_norm(deconv0)) if Bn else tf.nn.relu(deconv0)
#     deconv1 = deconv2d(deconv0, [self.batch_size, 7, 7, int(Cc/2)], f_h, f_w, 2, 2, bias=not Bn, padding='SAME')
#     deconv1 = tf.nn.relu(layers.batch_norm(deconv1)) if Bn else tf.nn.relu(deconv1)
#     deconv2 = deconv2d(deconv1, [self.batch_size, 14, 14, int(Cc/4)], f_h, f_w, 2, 2, bias=not Bn, padding='SAME')
#     deconv2 = tf.nn.relu(layers.batch_norm(deconv2)) if Bn else tf.nn.relu(deconv2)
#     deconv3 = deconv2d(deconv2, [self.batch_size, 28, 28, self.C], f_h, f_w, 2, 2, bias=not Bn, padding='SAME') #int(Cc/4)
       
    return G_prob

In [8]:
def discriminator(x):
#     D_h1 = tf.nn.relu(tf.matmul(x, D_W1) + D_b1)
#     #D_h2 = tf.nn.relu(tf.matmul(D_h1, D_W2) + D_b2)
#     D_logit = tf.matmul(D_h1, D_W3) + D_b3
#     D_prob = tf.nn.sigmoid(D_logit)
    
    D_h1 = tf.nn.leaky_relu(tf.nn.conv2d(x, DW_conv1, strides=[1, 2, 2, 1], padding='SAME') + Db_conv1)
    D_h2 = tf.nn.leaky_relu(layers.batch_norm(tf.nn.conv2d(D_h1, DW_conv2, strides=[1, 2, 2, 1], padding='SAME') + Db_conv2))
    D_h3 = tf.nn.leaky_relu(layers.batch_norm(tf.nn.conv2d(D_h2, DW_conv3, strides=[1, 2, 2, 1], padding='VALID') + Db_conv3))     
    #print D_h3.get_shape()
#     h_pool_flat = tf.reshape(D_h3,[-1,3*3*128])
#     D_h4 = tf.nn.relu(tf.matmul(h_pool_flat, DW_fc4) + Db_fc4)
#     D_logit = tf.matmul(D_h4, DW_fc5) + Db_fc5
# D_prob = tf.nn.sigmoid(D_logit)
    D_logit = tf.nn.conv2d(D_h3, DW_conv4, strides=[1, 2, 2, 1], padding='VALID') + Db_conv4
    D_prob = tf.nn.sigmoid(D_h3) #tf.reshape(D_h3, [-1,1])
    return D_prob, D_logit

In [9]:
G_sample = generator(Z)
D_real, D_logit_real = discriminator(tf.reshape(X,[-1,28,28,1]) )
D_fake, D_logit_fake = discriminator(tf.reshape(G_sample,[-1,28,28,1]))

(?, 14, 14, 16)
(?, 784)


In [10]:
# Alternative losses:
# -------------------
D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_real, labels=tf.ones_like(D_logit_real)))
D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.zeros_like(D_logit_fake)))
D_loss = D_loss_real + D_loss_fake
G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.ones_like(D_logit_fake)))

D_solver = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(D_loss, var_list=theta_D)
G_solver = tf.train.AdamOptimizer(learning_rate=0.001).minimize(G_loss, var_list=theta_G)

In [11]:
if not os.path.exists('out/'):
    os.makedirs('out/')

In [ ]:
with tf.Session() as sess:
    gpu_options = tf.GPUOptions()
    gpu_options.allow_growth=True 
    sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options,
        allow_soft_placement=True))
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())

    i = 0
    for it in range(50000):
        X_mb, _ = mnist.train.next_batch(mb_size)
        
        if it % 1000 == 0:
            samples = sess.run(G_sample, feed_dict={Z: sample_Z(16, Z_dim)})
            fig = plot(samples,4)
            plt.savefig('out/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')
            i += 1
            plt.close(fig)
        
            fig = plot(X_mb,12)
            plt.savefig('mini_batch/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')
            plt.close(fig)

        #print X_mb.shape

        _, D_loss_curr = sess.run([D_solver, D_loss], feed_dict={X: X_mb, Z: sample_Z(mb_size, Z_dim)})
        if it < 00:
            for it_G in range(5):
              _, G_loss_curr = sess.run([G_solver, G_loss], feed_dict={Z: sample_Z(mb_size, Z_dim)})
        else:
            _, G_loss_curr = sess.run([G_solver, G_loss], feed_dict={Z: sample_Z(mb_size, Z_dim)})


        if it % 1000 == 0:
            print('Iter: {}; D loss: {:.4};  G_loss: {:.4}'.format(it, D_loss_curr, G_loss_curr))
            save_model(None, saver, sess, it)

Iter: 0; D loss: 1.613;  G_loss: 1.683
Iter: 1000; D loss: 0.7623;  G_loss: 1.685
Iter: 2000; D loss: 1.015;  G_loss: 1.405
Iter: 3000; D loss: 1.07;  G_loss: 1.245
Iter: 4000; D loss: 1.319;  G_loss: 0.993
Iter: 5000; D loss: 1.322;  G_loss: 1.083
Iter: 6000; D loss: 1.108;  G_loss: 1.126
Iter: 7000; D loss: 1.219;  G_loss: 0.9769
Iter: 8000; D loss: 1.029;  G_loss: 1.205
Iter: 9000; D loss: 0.9653;  G_loss: 1.09
Iter: 10000; D loss: 0.8873;  G_loss: 1.189
Iter: 11000; D loss: 1.031;  G_loss: 1.312
Iter: 12000; D loss: 0.8579;  G_loss: 1.103
Iter: 13000; D loss: 0.8146;  G_loss: 1.105
Iter: 14000; D loss: 0.7862;  G_loss: 1.568
Iter: 15000; D loss: 0.7541;  G_loss: 1.644
Iter: 16000; D loss: 0.7084;  G_loss: 1.81


In [ ]:
with tf.Session() as sess:
    saver = tf.train.Saver()
    saver.restore(sess, "model/myckpt-49000")
#     sess.run(tf.initialize_all_variables())
#     print(sess.run(tf.global_variables()))
    
    # Returns all variables created with trainable=True in a var_list
    var_list = tf.trainable_variables()
    print("Trainable variables:------------------------")
    
    # 取出所有可训练参数的索引、形状和名称
    for idx, v in enumerate(var_list):
        print("param {:3}: {:15}   {}".format(idx, str(v.get_shape()), v.name))
        np.save('variable/'+str(idx)+'.npy',sess.run(var_list[idx].name))
#     print()
    #np.save("v1.npy",sess.run(v))

In [15]:
with tf.Session() as sess:
    saver = tf.train.Saver()
    saver.restore(sess, "model/myckpt-49000")
#     sess.run(tf.initialize_all_variables())
#     print(sess.run(tf.global_variables()))
    for i in range(10):
        samples = sess.run(G_sample, feed_dict={Z: sample_Z(100, Z_dim)})
        
        fig = plot(samples,10)
        plt.savefig('sample_gen/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')
        plt.close(fig)

INFO:tensorflow:Restoring parameters from model/myckpt-49000
